In [3]:
import boto3 
from automation.glue_job import GlueWrapper

## Load source data into Landing if needed

In [ ]:
# /home/glue_user/project_lf/ETL-TDD/temp/202305211851-claim-full.csv
# /home/glue_user/project_lf/ETL-TDD/temp/202305221132-policyholder-full.csv
# /home/glue_user/project_lf/ETL-TDD/temp/202305221136-provider-full.csv

!aws s3 cp /home/glue_user/project_lf/ETL-TDD/temp/202305211851-claim-full.csv s3://project-lf/etl/landing/claim_db/claim/full/202305211851-claim-full.csv

!aws s3 cp /home/glue_user/project_lf/ETL-TDD/temp/202305221132-policyholder-full.csv s3://project-lf/etl/landing/claim_db/policyholder/full/202305221132-policyholder-full.csv

!aws s3 cp /home/glue_user/project_lf/ETL-TDD/temp/202305221136-provider-full.csv s3://project-lf/etl/landing/claim_db/provider/full/202305221136-provider-full.csv

## Reset every staging tier except for Landing

In [56]:
!aws s3 ls s3://project-lf/etl/ --recursive | grep -v "landing" | awk '{print $4}' | xargs -I {} aws s3 rm s3://project-lf/etl/{}

delete: s3://project-lf/etl/etl/raw/


## Delete every glue job

In [76]:
# delete all glue ETL jobs
glue = boto3.client('glue', region_name='us-east-1')
jobs = glue.get_jobs()
for job in jobs['Jobs']:
    if job['Name'].startswith('stage_'):
        glue.delete_job(JobName=job['Name'])

## Refresh every .py job file in `code/jobs/` and `code/dependencies` 

In [82]:
# delete the libraries.zip file at s3://project-lf/code/libraries.zip
!aws s3 rm s3://project-lf/code/libraries.zip

# create a new zip file
!zip -r libraries.zip *

# write cli that copies libraries.zip to s3://project-lf/code/libraries.zip
!aws s3 cp libraries.zip s3://project-lf/code/libraries.zip

# delete everything but the libraries.zip file at s3://project-lf/code/
!aws s3 rm s3://project-lf/code/ --recursive --exclude "libraries.zip"

delete: s3://project-lf/code/libraries.zip


In [81]:
!ls 

automation	      run4_claim_fact.py
aws		      run_everything.py
errors		      stage_claim_into_access.py
etl		      stage_claim_into_optimised.py
interactive.ipynb     stage_claim_into_raw.py
libraries.zip	      stage_date_into_optimised.py
old		      stage_location_into_optimised.py
project-planning      stage_policyholder_into_access.py
__pycache__	      stage_policyholder_into_optimised.py
pytest.ini	      stage_policyholder_into_raw.py
redshift	      stage_procedure_into_optimised.py
run1_claim.py	      stage_provider_into_access.py
run1_date.py	      stage_provider_into_optimised.py
run1_policyholder.py  stage_provider_into_raw.py
run1_provider.py      stage_source_into_landing.py
run2_location.py      temp
run2_procedure.py     test_data
run3_policyholder.py  tests
run3_provider.py


## Upload .py job files to S3

In [23]:
import boto3

# Create an S3 client
s3_client = boto3.client('s3')

# Set the bucket and prefix
bucket = 'project-lf'
prefix = 'code/jobs/'

# Set the list of files to upload
files = [
    '/home/glue_user/project_lf/ETL-TDD/stage_claim_into_access.py',
    '/home/glue_user/project_lf/ETL-TDD/stage_claim_into_optimised.py',
    '/home/glue_user/project_lf/ETL-TDD/stage_claim_into_raw.py',
    '/home/glue_user/project_lf/ETL-TDD/stage_date_into_optimised.py',
    '/home/glue_user/project_lf/ETL-TDD/stage_location_into_optimised.py',
    '/home/glue_user/project_lf/ETL-TDD/stage_policyholder_into_access.py',
    '/home/glue_user/project_lf/ETL-TDD/stage_policyholder_into_optimised.py',
    '/home/glue_user/project_lf/ETL-TDD/stage_policyholder_into_raw.py',
    '/home/glue_user/project_lf/ETL-TDD/stage_procedure_into_optimised.py',
    '/home/glue_user/project_lf/ETL-TDD/stage_provider_into_access.py',
    '/home/glue_user/project_lf/ETL-TDD/stage_provider_into_optimised.py',
    '/home/glue_user/project_lf/ETL-TDD/stage_provider_into_raw.py',
    '/home/glue_user/project_lf/ETL-TDD/stage_source_into_landing.py'
]

# Upload each file to the S3 bucket
for file_path in files:
    # Get the file name
    file_name = file_path.split('/')[-1]

    # Set the object key
    key = f'{prefix}{file_name}'

    # Upload the file
    s3_client.upload_file(file_path, bucket, key)

print(f'Uploaded {len(files)} files to s3://{bucket}/{prefix}')

Uploaded 13 files to s3://project-lf/code/jobs/


## Initialise Glue session

In [77]:
# for each script in s3://project-lf/code/jobs/ create a glue job
import boto3

# Create an AWS Glue client
glue_client = boto3.client('glue', region_name='us-east-1')

# Set the parameters for the new Glue job
glue_role = 'data-quality-lf'


## Get a list of all script objects in `project-lf/code/jobs/`

In [47]:
# get all keys in s3://project-lf/code/jobs/
s3_resource = boto3.resource('s3')
bucket = 'project-lf'
prefix = 'code/jobs/'
objects = s3_resource.Bucket(bucket).objects.filter(Prefix=prefix)

## Create a job for every script object

In [78]:
glue_client = boto3.client('glue', region_name='us-east-1')
job_names = []
for s3_object in objects:
    print(s3_object.key)
    
    # Create an AWS Glue client

    # Set the parameters for the new Glue job
    glue_role = 'data-quality-lf'
    script_location = f's3://project-lf/{s3_object.key}'
    job_name = script_location.split('/')[-1].split('.')[0]
    extra_py_files = 's3://project-lf/code/dependencies/libraries.zip'
    glue_version = '4.0'
    additional_python_modules = "holidays"

    try:
        # Create the new Glue job
        response = glue_client.create_job(
            Name=job_name,
            Role=glue_role,
            Command={'Name': 'glueetl', 'ScriptLocation': script_location},
            GlueVersion=glue_version,
            DefaultArguments={
                '--extra-py-files': extra_py_files,
                '--additional-python-modules': additional_python_modules
            },
            WorkerType='G.1X',
            NumberOfWorkers=2
        )
        print(response)
    except Exception as e:
        print("Skipping job creation for", job_name, e)
    
    job_names.append(job_name)

code/jobs/stage_claim_into_access.py
{'Name': 'stage_claim_into_access', 'ResponseMetadata': {'RequestId': 'e7fe4482-0d24-4155-bc45-160c4ebe22f9', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 01 Jun 2023 14:36:50 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '34', 'connection': 'keep-alive', 'x-amzn-requestid': 'e7fe4482-0d24-4155-bc45-160c4ebe22f9'}, 'RetryAttempts': 0}}
code/jobs/stage_claim_into_optimised.py
{'Name': 'stage_claim_into_optimised', 'ResponseMetadata': {'RequestId': '69e2166a-fb6b-452f-9c2d-53dcca6c379c', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 01 Jun 2023 14:36:51 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '37', 'connection': 'keep-alive', 'x-amzn-requestid': '69e2166a-fb6b-452f-9c2d-53dcca6c379c'}, 'RetryAttempts': 0}}
code/jobs/stage_claim_into_raw.py
{'Name': 'stage_claim_into_raw', 'ResponseMetadata': {'RequestId': '58c136e5-a9b1-4858-b626-49c9b3f11790', 'HTTPStatusCode': 200, 'HTTPHeaders': {

## Define execution order of jobs

In [49]:
job_waves = [
     ['stage_claim_into_raw', 'stage_provider_into_raw', 'stage_policyholder_into_raw',
    'stage_date_into_optimised'],
     
    ['stage_claim_into_access','stage_provider_into_access',
        'stage_policyholder_into_access'],
    
    ['stage_location_into_optimised', 'stage_procedure_into_optimised'],
    
    ['stage_policyholder_into_optimised', 'stage_provider_into_optimised'],
    
    ['stage_claim_into_optimised']
]

## Define a function to run jobs and wait until they have completed

In [66]:
import boto3
import time
import colorama
from colorama import Fore, Style
colorama.init()

# Create an AWS Glue client
glue_client = boto3.client('glue', region_name='us-east-1')

# Function to run a list of jobs and wait for them to finish
def run_jobs(jobs):
    # Start each job and store its job run ID
    job_run_ids = {}
    for job_name in jobs:
        response = glue_client.start_job_run(JobName=job_name)
        job_run_id = response['JobRunId']
        job_run_ids[job_name] = job_run_id
        print(job_name, job_run_id)

    print("-"*80)
    
    # Wait for all jobs to finish
    while True:
        # Check the status of each job run
        all_finished = True
    
        for job_name, job_run_id in job_run_ids.items():
            try:
                response = glue_client.get_job(JobName=job_name)
                response = glue_client.get_job_run(JobName=job_name, RunId=job_run_id)
                status = response['JobRun']['JobRunState']
                if status not in ['SUCCEEDED', 'FAILED', 'STOPPED']:
                    all_finished = False
                else:
                    if status == 'SUCCEEDED':
                        print(Fore.GREEN + '\t' + job_name + 'has ' + status + '.' + Style.RESET_ALL)
                    elif status == 'FAILED':
                        error_message = response['JobRun'].get('ErrorMessage', 'No error message available')
                        print(Fore.RED + '\t' + job_name + 'has ' + status + '.' + Style.RESET_ALL)
                        print(Fore.RED + f'\t{job_name} has {status}. Error message: {error_message}' + Style.RESET_ALL)
                    else:
                        print(Fore.YELLOW + f'\t{job_name} has {status}.' + Style.RESET_ALL)
            except glue_client.exceptions.EntityNotFoundException:
                continue
        print("-"*80)
        # If all jobs have finished, exit the loop
        if all_finished:
            break
        # Otherwise, wait and check again
        time.sleep(10)

## Execute jobs in order

In [79]:
for count, job_set in enumerate(job_waves):
    print("*"*80)
    print(f"Starting on Run {count + 1}: {job_set}")
    print("*"*80)
    run_jobs(job_set)
print('All jobs finished')

********************************************************************************
Starting on Run 1: ['stage_claim_into_raw', 'stage_provider_into_raw', 'stage_policyholder_into_raw', 'stage_date_into_optimised']
********************************************************************************
stage_claim_into_raw jr_530bdd62405b4086720df8a48794583b1124fb9c614e789de3aea60f69abf258
stage_provider_into_raw jr_69ad2f15c2b84e16cef1459ff538c63f52816c12e8a079bc534ec79d4087c1cf
stage_policyholder_into_raw jr_1b11c7c13e90ea795649d170a355d84bf84f917650f35257e98a4c72f2de1f95
stage_date_into_optimised jr_e5e9d657454857d135aab706eb59fd4dd52edd5fc8fd5a24650fd7d269f2d683
--------------------------------------------------------------------------------
	Runnning stage_claim_into_raw... RUNNING
	Runnning stage_provider_into_raw... RUNNING
	Runnning stage_policyholder_into_raw... RUNNING
	Runnning stage_date_into_optimised... RUNNING
------------------------------------------------------------------------

In [46]:
# use aws cli to create landing folders
# make s3://project-lf/landing/claim_db/claim/full/ for claim, provider and policyholder
!aws s3api put-object --bucket project-lf --key etl/landing/claim_db/claim/full/
!aws s3api put-object --bucket project-lf --key etl/landing/claim_db/provider/full/
!aws s3api put-object --bucket project-lf --key etl/landing/claim_db/policyholder/full/

# cp source data files into landing folders


{
    "ETag": "\"d41d8cd98f00b204e9800998ecf8427e\"", 
    "ServerSideEncryption": "AES256"
}
{
    "ETag": "\"d41d8cd98f00b204e9800998ecf8427e\"", 
    "ServerSideEncryption": "AES256"
}
{
    "ETag": "\"d41d8cd98f00b204e9800998ecf8427e\"", 
    "ServerSideEncryption": "AES256"
}
